In [3]:
"""Calibrate LWR with Gipps"""
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import os
from flow.core.macroscopic import LWR
from flow.core.macroscopic.lwr import PARAMS as LWR_PARAMS
from flow.core.macroscopic.utils import run
from flow.core.macroscopic.Calibration.aggregate_micro_to_macro import agg_func
params = LWR_PARAMS.copy()

dir = "./Gipps/"

L = 2000
dx = 10
params['length'] = L
params['dx'] = dx
params['v_max_max'] = 100
params['rho_max_max'] = 300
params['total_time'] = 300
params["boundary_conditions"] = "loop"

Avg_pos = pd.DataFrame()
Avg_speed = pd.DataFrame()
file_list = os.listdir(dir)
for file in file_list:
    if "pos" in file:
        T = pd.read_csv(dir + file)
        Avg_pos = pd.concat([Avg_pos, T])
    if "speed" in file:
        S = pd.read_csv(dir + file)
        Avg_speed = pd.concat([Avg_speed, S])

## mean positions from csv
Pos = Avg_pos.groupby("time").mean()
## mean velocities from csv
Speeds = Avg_speed.groupby("time").mean()
# display(Pos.head())
# display(Speeds.head())


#single
## get densities amd velocities -> at specific time step t and t+1
t1 = 20
row = np.array(Pos.loc[t1])
speed = np.array(Speeds.loc[t1])
rho_init, vel_init = agg_func(row, speed, L, dx)
params['initial_conditions'] = rho_init

t2 = 21
row = np.array(Pos.iloc[t2])
speed = np.array(Speeds.iloc[t2])
rho_final, vel_final = agg_func(row, speed, L, dx)
##################################################################
# display(Pos.iloc[[t1]])
# display(Pos.iloc[[t2]])

# # #batch
# def sample_t(Pos, size=10):
#     return np.random.choice(Pos.index[:-1],size,replace=False)
# index = sample_t(Pos, size=3)
# next_index = index+1
# index,next_index
# matrix_initial = Pos.iloc[index]
# matrix_final = Pos.iloc[next_index]
# # display(matrix_initial.head())
# # display(matrix_final.head())
# # get densities amd velocities -> at specific time step t and t+1
# A = matrix_initial.apply(lambda row : agg_func(row, row, L, dx)[0], axis = 1)
# B = matrix_final.apply(lambda row : agg_func(row, row, L, dx)[0], axis = 1)

# Tensorflow

# Our Code

In [ ]:
# x values
rho_init

# y values
rho_final

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
params['rho_max'] =1
params['v_max'] = 11
params['CFL'] = 0.95
params['initial_conditions'] = rho_init
params['dt'] = 0.1

global my_params

def build_model(in_val): 
    global my_params

    my_params = tf.Variable(np.array([1, 11]), dtype=tf.float32)
    rho_max = my_params[0]
    v_max = my_params[1]
    CFL = 0.95
    
    env = LWR(params)
    env.reset()
    obs = env._step(in_val, v_max, rho_max, CFL) # one time step
    print(obs)
    
    return obs

input_ph = tf.placeholder(tf.float32, shape=(200,))

y_pred = build_model(input_ph)


TypeError: Input 'y' of 'Mul' Op has type bool that does not match type float32 of argument 'x'.

In [55]:
x = rho_init # T_1
y_true = rho_final # T_2

y_true_ph = tf.placeholder(tf.float32, shape=(200,))

loss = tf.reduce_mean(tf.square(y_pred - y_true_ph))
# optimizer -> computes of gradients and updates our variables of interest (beta in our case)
optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)
init = tf.global_variables_initializer()


with tf.Session() as session:
    session.run(init) # assign values to variables. Run to execute node.
    feed_dict = {input_ph: np.array(x), y_true_ph: np.array(y_true)} # static real data

    for _ in np.arange(30):
        #current parameters
        print(my_params.eval())
        #solve. Run to execute node.
        loss_val, _ = session.run([loss, optimizer], feed_dict)
        # current solve
        print(loss_val)

    # Plot results. Run to execute node.
    plt.scatter(x, y)
    plt.plot(x,session.run(y_pred,{input_ph:x}), "-r")
    plt.show()

[ 1. 11.]


ValueError: Cannot feed value of shape (200,) for Tensor 'Placeholder_39:0', which has shape '(?, 200)'

In [1]:
import tensorflow as tf
tf.__version__

'1.9.0'